## How to edit strategy plans in custom text

This notebook demonstrates how one can obtain politeness paraphrases on custom texts. This can be done in two ways:  

1. one can specify the desired strategy combinations to use for the given input 
2. alternatively, one could also specify the desired politeness level  

For option 2, we will use the model trained over averaged annotator scores on the Stanford Politeness Corpus (Wikipedia portion) to estimate politeness perceptions. Note that the coefficients obtained may not accurately reflect the politeness perceptions in other communication settings.  

In [2]:
import json
import spacy
from collections import defaultdict

from convokit import Corpus, Utterance, Speaker
from convokit import download
from convokit import PolitenessStrategies, TextParser

# you will need to update the path of the downloaded/trained model in settings.py first
from strategy_manipulation import remove_strategies_from_utt, add_strategies_to_utt

We will need spacy to help us parse the input texts, and strategy extraction is done via ConvoKit. 

In [11]:
# we use spacy to obtain parses for text, which the strategy extractor rely on 
spacy_nlp = spacy.load('en_core_web_sm', disable=['ner'])

# we use politeness strategy collection "politeness_local", 
# i.e., a subset of strategies that can be achieved through localized markers   
ps = PolitenessStrategies(strategy_attribute_name="strategies", \
                          marker_attribute_name="markers", \
                          strategy_collection="politeness_local")

### 1. Incorporating specified strategy plan 

We consider the following message as the running example: 

In [12]:
message = "Sorry to bother you, can you please proofread this article?"

Suppose we want to make the following request use the strategy plan: 

In [13]:
strategy_plan = {"Subjunctive", "For.Me", "Gratitude"}

We first identify and locate strategies currently used that are no longer part of the plan, delete the corresponding markers, and sequentially add strategies that need to be incorporated. This can be achieved as follows: 

In [14]:
def edit_utterance_by_plan(message, strategy_plan, spacy_nlp, politeness_transformer):
    
    # importantly, we need to have markers set to be true to know the exact positions of the markers for later edits
    utt = politeness_transformer.transform_utterance(message, markers=True)

    # strategies currently used
    strategy_set = {k for k,v in utt.meta['strategies'].items() if v == 1}
    
    utt.meta['strategy_set'] = strategy_plan
    
    # We can then determine strategies that needs to be deleted, as well as strategies that should be added, by comparing strategy_plan and strategy_set.
    to_delete = strategy_set - strategy_plan
    to_add = strategy_plan - strategy_set
    
    remove_strategies_from_utt(utt, to_delete, removed_attribute_name='context')
    
    return add_strategies_to_utt(utt, to_add, politeness_transformer, spacy_nlp)

In [15]:
edit_utterance_by_plan(message, strategy_plan, spacy_nlp, ps)

'could you proofread this article for me ? thanks .'

### 2. Paraphrasing for a target politeness level 

Alternatively, if one has good knowledge of the receiver's politeness perception model (we currently assume a simple linear regression model taking all local strategies as features), we can set up the ILP problem to generate the strategy combination to use, and then incorporate the strategy plan. 

In [16]:
from plan_with_ilp import get_ilp_solution

# load average perception model 
from settings import PERCEPTION_MODEL_PATH

with open(PERCEPTION_MODEL_PATH, 'r') as f:
    AVERAGE_MODEL = json.load(f)

In [17]:
def paraphrase_utterance_by_intention(message, intended_politeness, spacy_nlp, politeness_transformer, perception_model = AVERAGE_MODEL):
    
    # importantly, we need to have markers set to be true to know the exact positions of the markers for later edits
    utt = politeness_transformer.transform_utterance(message, markers=True)

    # strategies currently used
    strategy_set = {k for k,v in utt.meta['strategies'].items() if v == 1}
    
    utt.meta['strategy_set'] = strategy_set
    utt.meta['intended_politeness'] = intended_politeness
    
    # we assume perfect channel, and use the average model to approximate receiver perception 
    strategy_plan = get_ilp_solution('0', strategy_set, perception_model, perception_model, set(), intended_politeness=intended_politeness)
    print('Recommended strategy plan:', strategy_plan)
    
    to_delete = strategy_set - strategy_plan
    to_add = strategy_plan - strategy_set
    
    remove_strategies_from_utt(utt, to_delete, removed_attribute_name='context')
    
    return add_strategies_to_utt(utt, to_add, politeness_transformer, spacy_nlp)

We can then check how the message may be altered depending on the intended politeness level. (It is also worth noting that while strategies are planed out, the generation model is _not_ perfect in incorporating them.)

In [18]:
for intended_politeness in range(-1, 3):
    
    print("Intended politeness level = {}".format(intended_politeness))

    print(paraphrase_utterance_by_intention(message, intended_politeness, spacy_nlp, ps))
    print()

Intended politeness level = -1
Recommended strategy plan: {'By.The.Way', 'Swearing', 'Actually', 'Indicative'}
btw can you actually fucking proofread this article ?

Intended politeness level = 0
Recommended strategy plan: {'Please.Start', 'Indicative', 'For.Me', 'Conj.Start'}
please can you proofread this article ?

Intended politeness level = 1
Recommended strategy plan: {'Reassurance', 'Indicative'}
no problem . can you proofread this article ?

Intended politeness level = 2
Recommended strategy plan: {'For.You', 'Indicative', 'Greeting', 'Gratitude'}
hi . can you proofread this article for me ? thanks .

